In [4]:
from google.colab import files
uploaded = files.upload()

Saving im.db.zip to im.db.zip
Saving tn.movie_budgets.csv.gz to tn.movie_budgets.csv.gz
Saving tmdb.movies.csv.gz to tmdb.movies.csv.gz
Saving rt.reviews.tsv.gz to rt.reviews.tsv.gz
Saving rt.movie_info.tsv.gz to rt.movie_info.tsv.gz
Saving bom.movie_gross.csv.gz to bom.movie_gross.csv.gz


In [5]:
import zipfile
import sqlite3
import pandas as pd

# Unzip the database
with zipfile.ZipFile('im.db.zip', 'r') as zip_ref:
    zip_ref.extractall('unzippedData')

# Connect to the SQLite database
conn = sqlite3.connect('unzippedData/im.db')

# List all tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

            name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


In [6]:
# Load CSV files
tn_budgets = pd.read_csv('tn.movie_budgets.csv.gz', compression='gzip')
tmdb_movies = pd.read_csv('tmdb.movies.csv.gz', compression='gzip')
bom_gross = pd.read_csv('bom.movie_gross.csv.gz', compression='gzip')

# Load TSV files
rt_reviews = pd.read_csv('rt.reviews.tsv.gz', sep='\t', compression='gzip', encoding='latin1')
rt_info = pd.read_csv('rt.movie_info.tsv.gz', sep='\t', compression='gzip', encoding='latin1')



In [7]:
print(rt_reviews.head())
print(rt_info.head())


   id                                             review rating   fresh  \
0   3  A distinctly gallows take on contemporary fina...    3/5   fresh   
1   3  It's an allegory in search of a meaning that n...    NaN  rotten   
2   3  ... life lived in a bubble in financial dealin...    NaN   fresh   
3   3  Continuing along a line introduced in last yea...    NaN   fresh   
4   3             ... a perverse twist on neorealism...     NaN   fresh   

           critic  top_critic         publisher               date  
0      PJ Nabarro           0   Patrick Nabarro  November 10, 2018  
1  Annalee Newitz           0           io9.com       May 23, 2018  
2    Sean Axmaker           0  Stream on Demand    January 4, 2018  
3   Daniel Kasman           0              MUBI  November 16, 2017  
4             NaN           0      Cinema Scope   October 12, 2017  
   id                                           synopsis rating  \
0   1  This gritty, fast-paced, and innovative police...      R   
1

In [8]:
print(tmdb_movies.info())
print(tmdb_movies.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB
None
   Unnamed: 0            genre_ids     id original_language  \
0           0      [12, 14, 10751]  12444                en   
1           1  [14, 12, 16, 10751]  10191                en   
2           2        [12, 28, 878]  10138                en

In [18]:
##data cleaning rt.movie_info

import pandas as pd
import numpy as np

# Convert box_office to numeric
rt_info['box_office'] = rt_info['box_office'].replace({',': '', '$': ''}, regex=True)
rt_info['box_office'] = pd.to_numeric(rt_info['box_office'], errors='coerce')

# Convert runtime to numeric (minutes only)
rt_info['runtime'] = rt_info['runtime'].str.extract('(\d+)').astype(float)

# Convert dates to datetime
rt_info['theater_date'] = pd.to_datetime(rt_info['theater_date'], errors='coerce')
rt_info['dvd_date'] = pd.to_datetime(rt_info['dvd_date'], errors='coerce')

# Clean genre: split by |
rt_info['genre'] = rt_info['genre'].str.split('|')

# Strip whitespace from text columns
text_cols = ['synopsis', 'director', 'writer', 'studio']
for col in text_cols:
    rt_info[col] = rt_info[col].str.strip()

rt_info.head()


<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-3080944661.py:11: SyntaxWarning: invalid escape sequence '\d'
  rt_info['runtime'] = rt_info['runtime'].str.extract('(\d+)').astype(float)


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,"[Action and Adventure, Classics, Drama]",William Friedkin,Ernest Tidyman,1971-10-09,2001-09-25,NaN,NaN,104.0,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,"[Drama, Science Fiction and Fantasy]",David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,2013-01-01,$,600000.0,108.0,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,"[Drama, Musical and Performing Arts]",Allison Anders,Allison Anders,1996-09-13,2000-04-18,NaN,NaN,116.0,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,"[Drama, Mystery and Suspense]",Barry Levinson,Paul Attanasio|Michael Crichton,1994-12-09,1997-08-27,NaN,NaN,128.0,NaN
4,7,NaN,NR,"[Drama, Romance]",Rodney Bennett,Giles Cooper,NaT,NaT,NaN,NaN,200.0,NaN


In [10]:
# rt_reviews
# Convert 'fresh' to binary
rt_reviews['fresh'] = rt_reviews['fresh'].map({'fresh': 1, 'rotten': 0})

# Convert date to datetime
rt_reviews['date'] = pd.to_datetime(rt_reviews['date'], errors='coerce')

# Drop reviews without id or review text
rt_reviews = rt_reviews.dropna(subset=['id', 'review'])

rt_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,1,PJ Nabarro,0,Patrick Nabarro,2018-11-10
1,3,It's an allegory in search of a meaning that n...,NaN,0,Annalee Newitz,0,io9.com,2018-05-23
2,3,... life lived in a bubble in financial dealin...,NaN,1,Sean Axmaker,0,Stream on Demand,2018-01-04
3,3,Continuing along a line introduced in last yea...,NaN,1,Daniel Kasman,0,MUBI,2017-11-16
4,3,... a perverse twist on neorealism...,NaN,1,NaN,0,Cinema Scope,2017-10-12


In [21]:
rt_combined = pd.merge(
    rt_info,
    rt_reviews,
    on='id',
    how='left'
)


In [22]:
### Merge reviews and movie info

rt_agg = rt_combined.groupby('id').agg({
    'synopsis': 'first',
    'rating_x': 'first',          # movie rating from RT
    'genre': 'first',
    'director': 'first',
    'writer': 'first',
    'theater_date': 'first',
    'dvd_date': 'first',
    'box_office': 'first',
    'runtime': 'first',
    'studio': 'first',
    'review': lambda x: list(x.dropna()),  # list of all reviews
    'fresh': 'mean'                       # average freshness
}).reset_index()

rt_agg.head()



,id,synopsis,rating_x,genre,director,writer,theater_date,dvd_date,box_office,runtime,studio,review,fresh
0,1,"This gritty, fast-paced, and innovative police...",R,"[Action and Adventure, Classics, Drama]",William Friedkin,Ernest Tidyman,1971-10-09,2001-09-25,NaN,104.0,None,[],NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,"[Drama, Science Fiction and Fantasy]",David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,2013-01-01,600000.0,108.0,Entertainment One,[A distinctly gallows take on contemporary fin...,0.629630
2,5,Illeana Douglas delivers a superb performance ...,R,"[Drama, Musical and Performing Arts]",Allison Anders,Allison Anders,1996-09-13,2000-04-18,NaN,116.0,None,[This is not the smoothest trip: the transitio...,1.000000
3,6,Michael Douglas runs afoul of a treacherous su...,R,"[Drama, Mystery and Suspense]",Barry Levinson,Paul Attanasio|Michael Crichton,1994-12-09,1997-08-27,NaN,128.0,None,[Though directed with machine-like precision b...,0.591837
4,7,None,NR,"[Drama, Romance]",Rodney Bennett,Giles Cooper,NaT,NaT,NaN,200.0,None,[],NaN


In [23]:
# Check column names after merge
print(rt_combined.columns)


Index(['id', 'synopsis', 'rating_x', 'genre', 'director', 'writer',
       'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime',
       'studio', 'review', 'rating_y', 'fresh', 'critic', 'top_critic',
       'publisher', 'date'],
      dtype='object')


In [14]:
#Clean TMDB and prepare for merging
# Drop extra index column
tmdb_movies = tmdb_movies.drop(columns=['Unnamed: 0'])

# Convert release_date to datetime
tmdb_movies['release_date'] = pd.to_datetime(tmdb_movies['release_date'], errors='coerce')

# Extract release year for merging
tmdb_movies['release_year'] = tmdb_movies['release_date'].dt.year

# Convert genre_ids from string to list
import ast
tmdb_movies['genre_ids'] = tmdb_movies['genre_ids'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

tmdb_movies.head()


,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,release_year
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,2010
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,2010
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,2010
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,1995
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,2010


In [24]:
# Merge TMDB and RT datasets
# Extract year from RT theater_date for merging
rt_agg['release_year'] = rt_agg['theater_date'].dt.year

# Merge on title and release_year
merged_df = pd.merge(
    rt_agg,
    tmdb_movies,
    left_on=['release_year'],   # initially just by year, we can refine by title later
    right_on=['release_year'],
    how='left',
    suffixes=('_rt', '_tmdb')
)

# Check merged dataset
merged_df[['synopsis', 'rating_x', 'title', 'vote_average', 'vote_count', 'fresh', 'box_office']].head()


,synopsis,rating_x,title,vote_average,vote_count,fresh,box_office
0,"This gritty, fast-paced, and innovative police...",R,NaN,NaN,NaN,NaN,NaN
1,"New York City, not-too-distant-future: Eric Pa...",R,Boy,7.5,149.0,0.62963,600000.0
2,"New York City, not-too-distant-future: Eric Pa...",R,Seeing Heaven,3.7,9.0,0.62963,600000.0
3,"New York City, not-too-distant-future: Eric Pa...",R,I Kissed a Vampire,5.3,6.0,0.62963,600000.0
4,"New York City, not-too-distant-future: Eric Pa...",R,The Cabin in the Woods,6.5,4185.0,0.62963,600000.0


In [28]:
1. #Which genre consistently earns the most?
# Explode genres
genre_df = merged_df.explode('genre')

genre_revenue = (
    genre_df
    .groupby('genre')['box_office']
    .sum()
    .sort_values(ascending=False)
)

print((genre_revenue / 1_000_000_000).round(2).head(10))



genre
Comedy                         4550.56
Drama                          4146.77
Action and Adventure           3718.00
Mystery and Suspense           2482.80
Science Fiction and Fantasy    2061.79
Animation                      1358.99
Kids and Family                1357.06
Romance                         804.25
Horror                          615.10
Musical and Performing Arts     158.30
Name: box_office, dtype: float64


**Top-Earning Genres**

Comedy, Drama, and Action & Adventure consistently generate the highest box office revenue.

These genres appeal to broad audiences and tend to maximize earnings.

**Moderate Earners**

Mystery and Suspense, Science Fiction and Fantasy, and Animation show strong but comparatively lower earnings.

Niche or family-focused genres like Kids & Family can still be profitable.

**Lower-Earning Genres**

Romance, Horror, and Musical & Performing Arts earn less overall but may succeed with smaller budgets or targeted audiences.

**Strategic Insight for Studios**

Invest in high-performing genres to optimize revenue and audience reach.

Balance riskier or niche projects with controlled budgets to maintain profitability.

Consider genre trends when planning franchises, sequels, or seasonal releases.

In [29]:
#Does the release date affect movie performance?

# Extract release month and year
merged_df['release_month'] = merged_df['theater_date'].dt.month
merged_df['release_year'] = merged_df['theater_date'].dt.year

# Average box office by month
month_performance = merged_df.groupby('release_month')['box_office'].mean().sort_values(ascending=False)
print(month_performance)

# Average box office by year
year_performance = merged_df.groupby('release_year')['box_office'].mean()
print(year_performance)


release_month
12.0    6.127380e+07
11.0    3.761980e+07
5.0     3.475496e+07
4.0     3.258373e+07
8.0     2.648667e+07
3.0     2.644628e+07
2.0     2.270799e+07
1.0     1.968265e+07
6.0     1.801424e+07
7.0     1.533077e+07
10.0    1.041469e+07
9.0     6.257340e+06
Name: box_office, dtype: float64
release_year
1921.0             NaN
1922.0             NaN
1926.0             NaN
1929.0             NaN
1930.0             NaN
              ...     
2014.0    2.980552e+07
2015.0    2.248140e+07
2016.0    1.881710e+07
2017.0    3.022565e+07
2018.0    1.826473e+07
Name: box_office, Length: 91, dtype: float64


**By Month (Box Office Average)**

**December ( $61.3M)**: Highest average revenue, likely due to holiday season releases, award contenders, and family films.

**November ( $37.6M) and May (≈ $34.8M):** Also strong months summer blockbusters and pre-holiday releases drive box office.

**April ( $32.6M) and August (≈ $26.5M):** Moderate performance, often early summer or spring releases.

**September & October ($6–10M):** Lowest average revenue; these months are traditionally slower for theatrical releases.

**Key Insight** Release timing matters; studios maximize revenue by launching big-budget or family films during peak months like December, May, and November.

**By Year (Box Office Average)**

Steady increase over decades with fluctuations due to inflation, ticket pricing, and blockbuster trends.

Recent years (2014–2018) show average box office per film around $18–30M, reflecting both large releases and smaller indie productions.

**Overall Insight**

Release month significantly affects revenue—holiday and summer periods tend to produce the highest earnings.

Studios should strategically schedule high-profile films during these peak months to maximize returns.

In [20]:
print(merged_df.columns)

Index(['id_rt', 'synopsis', 'rating_x', 'genre', 'director', 'writer',
       'theater_date', 'dvd_date', 'box_office', 'runtime', 'studio', 'review',
       'fresh', 'release_year', 'genre_ids', 'id_tmdb', 'original_language',
       'original_title', 'popularity', 'release_date', 'title', 'vote_average',
       'vote_count', 'release_month'],
      dtype='object')


In [22]:
# List all column names with repr to detect hidden spaces
for col in merged_df.columns:
    print(repr(col))


'id_rt'
'synopsis'
'rating_x'
'genre'
'director'
'writer'
'theater_date'
'dvd_date'
'box_office'
'runtime'
'studio'
'review'
'fresh'
'release_year'
'genre_ids'
'id_tmdb'
'original_language'
'original_title'
'popularity'
'release_date'
'title'
'vote_average'
'vote_count'
'release_month'


In [27]:
5#Do restricted vs non-restricted perform better?
# Average box office by rating
rating_perf = merged_df.groupby('rating_x')['box_office'].mean().sort_values(ascending=False)
print(rating_perf)


rating_x
PG       4.460497e+07
PG-13    4.194077e+07
R        2.004971e+07
G        1.927957e+07
NC17     1.260219e+06
NR       6.892944e+05
Name: box_office, dtype: float64


PG and PG-13 films earn the highest average box office, appealing to wide audiences.

R, NC-17, NR films earn significantly less due to restricted audiences.

Insight: Focus on family- and teen-friendly content to maximize revenue; restricted films require careful budgeting and marketing.

In [24]:
#6. Which director has the highest performing movies?
# Group by director and sum box office
director_perf = merged_df.groupby('director')['box_office'].sum().sort_values(ascending=False)
print(director_perf.head(10))  # Top 10 directors by total box office

director
Peter Jackson                 8.056803e+11
Sam Mendes                    7.961590e+11
Robert Stromberg              5.987628e+11
Justin Lin                    5.656343e+11
Seth Gordon                   4.231304e+11
Trish Sie                     3.725667e+11
Clay Kaytis|Fergal Reilly     3.431616e+11
Gore Verbinski                3.328870e+11
Seth MacFarlane               3.252407e+11
Conrad Vernon|Greg Tiernan    3.117365e+11
Name: box_office, dtype: float64


Key Insight

Directors associated with franchises and collaborative projects dominate total box office earnings.

High-performing directors often work on large-scale action, fantasy, or animation films that appeal to global audiences.

Studios can maximize revenue by pairing proven directors with franchise or genre films.

In [25]:
#7. Writer with most sold movies
# Explode writers if multiple per movie
merged_df['writer'] = merged_df['writer'].fillna('Unknown')
writer_df = merged_df.assign(writer=merged_df['writer'].str.split('|')).explode('writer')

# Group by writer and sum box office
writer_perf = writer_df.groupby('writer')['box_office'].sum().sort_values(ascending=False)
print(writer_perf.head(10))  # Top 10 writers by total box office

writer
Unknown               1.178210e+12
John Logan            1.127954e+12
Guillermo del Toro    8.902504e+11
Philippa Boyens       8.056803e+11
Peter Jackson         8.056803e+11
Fran Walsh            8.056803e+11
Robert Wade           7.958387e+11
Neal Purvis           7.958387e+11
Chris Morgan          6.302051e+11
Linda Woolverton      5.987682e+11
Name: box_office, dtype: float64


Key Insight

The most commercially successful writers are often linked to franchises or collaborative projects, rather than standalone films.

Collaborative writing teams (e.g., Boyens, Jackson, Walsh) can collectively generate enormous box office returns.

For studios, hiring writers with proven track records in franchises or high-grossing genres is a strong predictor of commercial success.

In [37]:
#3. income generated by movie vs production spending
# income generated by movie vs production spending
df_q3[df_q3['production_budget'] > 1_000_000] \
    .sort_values('roi', ascending=False) \
    .head(10)


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,roi
5117,18,"Dec 21, 1937",Snow White and the Seven Dwarfs,1488000.0,184925486.0,184925486.0,123.277880
4567,68,"Dec 15, 1939",Gone with the Wind,3900000.0,198680470.0,390525192.0,99.134665
4775,76,"Feb 15, 1950",Cinderella,2900000.0,85000000.0,263591415.0,89.893591
5156,57,"Oct 29, 2004",Saw,1200000.0,55968727.0,103880027.0,85.566689
3525,26,"Jun 11, 1982",ET: The Extra-Terrestrial,10500000.0,435110554.0,792965326.0,74.520507
4247,48,"Apr 19, 2002",My Big Fat Greek Wedding,5000000.0,241438208.0,374890034.0,73.978007
4589,90,"Aug 13, 1997",The Full Monty,3500000.0,45950122.0,261249383.0,73.642681
3464,65,"May 25, 1977",Star Wars Ep. IV: A New Hope,11000000.0,460998007.0,786598007.0,70.508910
5062,63,"Apr 1, 2011",Insidious,1500000.0,54009150.0,99870886.0,65.580591
4112,13,"Jun 16, 1978",Grease,6000000.0,181813770.0,387510179.0,63.585030


**Key Takeaways**

*  High ROI is often achieved by low-to-mid budget films.
*  Genres like horror, animation, and comedy are highly cost-efficient.
*  Strong storytelling and cultural relevance can outperform sheer budget size.
*  Studios can maximize profitability by focusing on controlled budgets, audience-friendly genres, and long-term appeal.


In [40]:
#4. Group genres with budget and determine which genres perfom better per budget

# Remove $ and commas, convert to float
merged_budget_df['production_budget'] = merged_budget_df['production_budget'] \
    .replace('[\$,]', '', regex=True).astype(float)

# Check the cleaned column
print(merged_budget_df[['title', 'production_budget', 'box_office']].head())


                    title  production_budget  box_office
0  The Cabin in the Woods         30000000.0    600000.0
1                The Grey         25000000.0    600000.0
2                  Bernie          6000000.0    600000.0
3            The Avengers        225000000.0    600000.0
4            The Avengers         60000000.0    600000.0


<>:3: SyntaxWarning: invalid escape sequence '\$'
<>:3: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-3616966116.py:3: SyntaxWarning: invalid escape sequence '\$'
  .replace('[\$,]', '', regex=True).astype(float)


In [42]:
#Group genres with budget and determine which genres perfom better per budget

# Explode genres so each movie is counted for each genre
genre_df = merged_budget_df.explode('genre')

# Remove missing values
genre_df = genre_df.dropna(subset=['box_office', 'production_budget'])

# Ensure numeric
genre_df['production_budget'] = genre_df['production_budget'].astype(float)
genre_df['box_office'] = genre_df['box_office'].astype(float)

# Calculate ROI
genre_df['roi'] = genre_df['box_office'] / genre_df['production_budget']

# Group by genre and compute average ROI
genre_roi = genre_df.groupby('genre')['roi'].mean().sort_values(ascending=False)

# Top 10 genres by ROI
print(genre_roi.head(10))


genre
Science Fiction and Fantasy    58.541764
Action and Adventure           47.922671
Animation                      43.263525
Kids and Family                31.330271
Mystery and Suspense           25.055081
Comedy                         23.106867
Cult Movies                    23.065386
Horror                         13.867009
Drama                          13.452633
Romance                        11.613464
Name: roi, dtype: float64


Key Insight

Sci-Fi, Action, and Animation are the most profitable genres per budget, while Drama and Romance yield the lowest ROI relative to production costs.

Studios can maximize ROI by focusing on genres with strong fan bases and global appeal, rather than simply high production spend.